In [ ]:
!pip install rasterio gdown tqdm

In [2]:
import rasterio
import numpy as np
import gdown
from tqdm import tqdm

# URL do arquivo
url = "https://drive.google.com/uc?id=1BrtGByxXu5xqBGOvSBmPWkb_YGZagHav"
output = "image.tif"
gdown.download(url, output, quiet=False)

with rasterio.open(output) as src:
    # Primeiro, contamos o número total de blocos para o progresso:
    total_blocks = sum(1 for _ in src.block_windows(1))

    # Contadores
    total_pixels = 0
    no_data_pixels = 0
    soja_pixels = 0
    pastagem_pixels = 0

    # Itera sobre os blocos com barra de progresso
    for ji, window in tqdm(src.block_windows(1), total=total_blocks, desc="Processando blocos"):
        # Lê somente o bloco atual
        data = src.read(1, window=window)

        # Atualiza contadores
        total_pixels += data.size
        no_data_pixels += np.count_nonzero(data == 0)
        soja_pixels += np.count_nonzero(data == 39)
        pastagem_pixels += np.count_nonzero(data == 15)

# Pixels válidos (sem dados excluídos)
valid_pixels = total_pixels - no_data_pixels

# Evitar divisão por zero caso não haja pixels válidos
if valid_pixels > 0:
    perc_soja = (soja_pixels / valid_pixels) * 100
    perc_pastagem = (pastagem_pixels / valid_pixels) * 100
else:
    perc_soja = 0
    perc_pastagem = 0

# Área do Brasil em hectares (IBGE)
area_brasil_ha = 851576700

# Áreas correspondentes
area_soja_ha = (perc_soja / 100) * area_brasil_ha
area_pastagem_ha = (perc_pastagem / 100) * area_brasil_ha

# Impressão dos resultados
print("\nResultados:")
print(f"1. Quantidade total de pixels: {total_pixels}")
print(f"2. Quantidade de pixels sem dados (código 0): {no_data_pixels}")
print(f"3. Quantidade de pixels de soja (código 39): {soja_pixels}")
print(f"4. Quantidade de pixels de pastagem (código 15): {pastagem_pixels}")

print("\nCálculos de área (considerando proporção em relação à área do Brasil):")
print(f"Área de plantio de soja: {area_soja_ha:.2f} hectares, Pixels: {soja_pixels}")
print(f"Área de pastagem: {area_pastagem_ha:.2f} hectares, Pixels: {pastagem_pixels}")

Downloading...
From (original): https://drive.google.com/uc?id=1BrtGByxXu5xqBGOvSBmPWkb_YGZagHav
From (redirected): https://drive.google.com/uc?id=1BrtGByxXu5xqBGOvSBmPWkb_YGZagHav&confirm=t&uuid=a9da2fb3-7db3-445b-82f9-eeaa6d274ac4
To: /content/image.tif
100%|██████████| 1.07G/1.07G [00:10<00:00, 100MB/s]
Processando blocos: 100%|██████████| 94240/94240 [01:28<00:00, 1067.97it/s]


Resultados:
1. Quantidade total de pixels: 24599016701
2. Quantidade de pixels sem dados (código 0): 14814537822
3. Quantidade de pixels de soja (código 39): 468443516
4. Quantidade de pixels de pastagem (código 15): 1895700288

Cálculos de área (considerando proporção em relação à área do Brasil):
Área de plantio de soja: 40770243.20 hectares, Pixels: 468443516
Área de pastagem: 164989287.16 hectares, Pixels: 1895700288
